### Import packages

In [8]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import time

In [9]:
# Function to create the CNN model
def get_model(bp, nunits):
    model = Sequential()
    model.add(Embedding(input_dim=bp, output_dim=128, input_length=bp))
    model.add(Conv1D(filters=32, kernel_size=9, activation="relu"))
    model.add(MaxPooling1D(pool_size=5))
    model.add(Conv1D(filters=32, kernel_size=9, activation="relu"))
    model.add(Flatten())
    model.add(Dense(units=1, activation="sigmoid"))
    
    model.compile(
        optimizer=RMSprop(),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [6]:
# Function to read and preprocess FASTA file
def read_fasta(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
    
    data = []
    for i in range(0, len(lines), 2):
        seq_id = lines[i].strip()[1:]  # Remove '>' from the header
        seq = lines[i+1].strip()
        data.append((seq_id, seq))
    
    return data

# Function to convert sequences to numerical format
def seq_to_num(seqs, bp):
    char_to_num = {
        "A": 1, "a": 1, "B": 2, "b": 2, "C": 3, "c": 3,
        "D": 4, "d": 4, "E": 5, "e": 5, "F": 6, "f": 6,
        "G": 7, "g": 7, "H": 8, "h": 8, "I": 9, "i": 9,
        "J": 10, "j": 10, "K": 11, "k": 11, "L": 12, "l": 12,
        "M": 13, "m": 13, "N": 14, "n": 14, "O": 15, "o": 15,
        "P": 16, "p": 16, "Q": 17, "q": 17, "R": 18, "r": 18,
        "S": 19, "s": 19, "T": 20, "t": 20, "U": 21, "u": 21,
        "V": 22, "v": 22, "W": 23, "w": 23, "X": 24, "x": 24,
        "Y": 25, "y": 25, "Z": 26, "z": 26, ".": 27, "#": 28,
        "~": 29, "*": 30
    }
    
    seqs_num = []
    for seq in seqs:
        seq_num = [char_to_num.get(char, 0) for char in seq]
        seqs_num.append(seq_num)
    
    return np.array(seqs_num)

In [10]:
# Main script
input_file = "ttc.fasta"
output_file = input_file.replace(".fasta", ".cnn.output.txt")

In [11]:
# Read and preprocess data
data = read_fasta(input_file)
data_labels = np.array([int(seq_id.split("_")[-1]) for seq_id, _ in data])
data_seqs = [seq for _, seq in data]

In [14]:
print(data_seqs)

['...']


In [15]:
# Convert sequences to numerical format
bp = 240  # NRTI sequence length
seqs_num = seq_to_num(data_seqs, bp)

In [16]:
# Pad sequences
data_f = pad_sequences(seqs_num, padding="post", maxlen=bp)

In [24]:
# Initialize lists to store results
validation_scores = []  # To store accuracy for each fold
eval_list = []  # To store evaluation results for each fold
rocout_list = []  # To store ROC outputs for each fold

# Set up 5-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=1234)

In [22]:
# Function to calculate performance metrics
def perf_measure(y_ref, y_pred):
    TP = np.sum((y_ref == 1) & (y_pred == 1))
    TN = np.sum((y_ref == 0) & (y_pred == 0))
    FP = np.sum((y_ref == 0) & (y_pred == 1))
    FN = np.sum((y_ref == 1) & (y_pred == 0))
    return TP, FP, TN, FN

In [ ]:
# Start cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(data_f)):
    print(f"Processing fold {fold + 1}...")
    
    # Split data into training and validation sets
    train_data, val_data = data_f[train_idx], data_f[val_idx]
    train_labels, val_labels = data_labels[train_idx], data_labels[val_idx]
    
    # Calculate class weights
    zero = np.sum(train_labels == 0)
    one = np.sum(train_labels == 1)
    weight_0 = 1
    weight_1 = zero / one
    
    # Create and train the model
    model = get_model(bp, nunits=99)
    model.fit(
        train_data,
        train_labels,
        epochs=20,
        batch_size=64,
        class_weight={0: weight_0, 1: weight_1},
        verbose=1
    )
    # Evaluate the model on the validation set
    results = model.evaluate(val_data, val_labels, verbose=0)
    validation_scores.append(results[1])  # Store accuracy
    print(f"Results for fold {fold + 1}: {results}")
    
    # Predict classes and calculate performance metrics
    val_preds = (model.predict(val_data) > 0.5).astype(int)
    TP, FP, TN, FN = perf_measure(val_labels, val_preds)
    print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")
    
    # Save evaluation results to a dictionary
    eval_results = {
        "fold": fold + 1,
        "accuracy": results[1],
        "loss": results[0],
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN
    }
    eval_list.append(eval_results)
    
    # ROC output
    preds_proba = model.predict(val_data)
    rocout = np.column_stack((preds_proba, val_preds, val_labels))
    rocout_list.append(rocout)

In [26]:
# Save evaluation results to a CSV file
eval_df = pd.DataFrame(eval_list)
eval_df.to_csv(f"{input_file}.eval_results.csv", index=False)

In [27]:
# Save ROC outputs to separate CSV files
for fold, rocout in enumerate(rocout_list):
    roc_df = pd.DataFrame(rocout, columns=["Predicted_Probability", "Predicted_Class", "True_Label"])
    roc_df.to_csv(f"{input_file}.roc.fold_{fold + 1}.csv", index=False)